# AUA, DS 229 – MLOps
### Week 5 – Special topics in data processing and model development

***

Agenda:
- Processing categorical and numerical variables
- Handling missing data
- Hyperparameter optimization with Optuna
- Quantization of DNN

In [ ]:
# !pip install category-encoders
# !pip install optuna
# !pip install torchvision
# !pip install xgboost
# !pip install plotly==5.13.0

### **Categorical variable encoding**

In data science, before starting the modeling process, data preparation is a necessary step. There are several tasks that need to be performed during this stage, one of which is encoding categorical data. This is considered critical because in real life, most of the data comes in the form of categorical string values, while most machine learning models only work with integer or other numerical values that can be understood by the model. The reason for this is that mathematical operations, which are at the heart of all models, can only be performed using numerical data, not strings or any other data types. The numbers used can be either floating-point or integer values.

Encoding categorical data is a process of converting categorical data into integer format so that the data with converted categorical values can be provided to the models. In this section, we will discuss categorical data encoding algorithms and will try to understand why we need the process of categorical data encoding. 

Categorical variables can be divided into two categories: **Nominal (no particular order)** and **Ordinal (some ordered)**.
<center><img src="images/categorical_data.png" width="900" height="900"/></center>

In [ ]:
import pandas as pd
import numpy as np


original_data = pd.DataFrame({
    "color": ["red", "red", "blue", "orange", "red", "blue", "blue", "red", "orange", "blue"],
    "condition": ["Very good", "Good", "Medium", "Very good", "Bad", "Medium", "Bad", "Bad", "Medium", "Good"],
    "mileage": [120000, 90000, 150000, 200000, 130000, 145000, 95000, 70000, 105000, 65000],
    "price": [10.1, 8.5, 6.3, 9.0, 12.2, 14.5, 10.8, 11.8, 12.4, 7.5]
})

original_data.head(n=10)

#### 1) Label Encoding

Under this encoding scheme, every category is given a numerical value ranging from 1 to N (where N is the total number of categories for that feature). However, a key drawback of this method is that there is no inherent relationship or order between the categories (assigned labels), though a ML algorithm may mistakenly interpret them as having some form of order or connection. For example, in the case of attribute **color** we may have `"red" < "blue" < "orange"` with the corresponding assignment of labels 1, 2 and 3. 

For the implementation either use `factorize` function from **pandas** or `LabelEncoder` from **sklearn**.

In [ ]:
data = original_data.copy()

In [ ]:
data.loc[:, "color_labels"] = pd.factorize(data["color"])[0].reshape(-1, 1)
data

#### 2) One Hot Encoding

This approach involves assigning a vector consisting of 1s and 0s to each category, representing the presence or absence of that feature. The number of vectors required corresponds to the number of categories for the features. However, if there are a large number of categories for a particular feature, this can result in a significant slowdown in the learning process due to the large number of resulting columns.

The desired outcome can be achieved using either `get_dummies` function from **pandas** or `OneHotEncoder` from **sklearn**.

In [ ]:
data = original_data.copy()

In [ ]:
pd.get_dummies(data, prefix=["color"], columns=["color"])

#### 3) Ordinal Encoding

Ordinal encoding is used to maintain the ordinal nature of variables in their encoding process, but it should only be applied to variables that are inherently ordinal. This encoding method is similar to label encoding, but it differs in that it takes into account the ordinal nature of the variable, whereas label encoding does not make this distinction and simply assigns a sequence of integers.

For the implementation, one needs to define the mapping between categories and labels manually. Labels can be assigned using `pandas.Series.map` method.

In [ ]:
data = original_data.copy()

In [ ]:
temperature_mapping = {
    "Bad": 1, 
    "Medium": 2, 
    "Good": 3, 
    "Very good": 4
}
data["condition_labels"] = data["condition"].map(temperature_mapping)
data

#### 4) Binary Encoding

Binary encoding transforms a category into a series of binary digits, with each digit creating a separate feature column. The number of resulting features generated by binary encoding is equal to $$⌈\log_2 n⌉,$$ where $n$ represents the number of unique categories. In the given example, there are four features, which would result in three binary-encoded features. In contrast to One Hot Encoding, binary encoding requires fewer feature columns. For instance, if there were 100 categories, One Hot Encoding would require 100 features, while binary encoding would only require seven features. There are 3 steps to apply the encoding:
1) The categories are first converted to numeric order starting from 1 (order is created as categories appear in the data and **do not mean any ordinal nature**).
2) The assigned integers are converted into binary code, so for example, 3 becomes 11.
3) Then the digits of the binary number form separate columns.

For the implementation we will use [category_encoders](https://github.com/scikit-learn-contrib/category_encoders) package.

In [ ]:
data = original_data.copy()

In [ ]:
from category_encoders import BinaryEncoder


encoder = BinaryEncoder(cols=["color"])
color_bin = encoder.fit_transform(data["color"])
data = pd.concat([data, color_bin], axis=1)
data

#### 5) Mean Encoding

Mean encoding shares similarities with label encoding, but instead of assigning labels arbitrarily, they are directly linked to the target variable. In this encoding method, the label assigned to each category is determined by the mean value of the target variable in the training data for that category. Mean encoding helps to reveal relationships between similar categories, but these relationships are limited to the categories and the target variable. The benefits of mean encoding include its ability to expedite the learning process and its minimal impact on data volume. However, mean encoding is often associated with overfitting and requires regularization techniques such as cross-validation. To apply a mean encoding to a categorcial variable first of all one needs to group by that categorical variable and pick the mean of 'target' fo each category.

In [ ]:
data = original_data.copy()

In [ ]:
target = "price"  # Consider the target to be 'price'.

mean_enc_map = data.groupby("color")[target].mean()
data.loc[:, "color_enc"] = data["color"].map(mean_enc_map)
data

There are other encodings, such as **Probability Ratio Encoding**, **James-Stein Encoding**, **M-estimator Encoding**, etc.

### **Numerical variable encoding**

When you're processing data, it's crucial to ensure that your features are within comparable ranges, which is called feature scaling. This is a straightforward task that can significantly enhance your model's performance, and failing to do so can lead to nonsensical predictions, particularly when using traditional algorithms such as gradient-boosted trees and logistic regression.

#### 1) Min-max scaling

A simple way to scale your features is to get each feature to be in the range $[0, 1]$. Given a variable $x$, its values can be rescaled to be in this range using the following formula:
$$
x' = \frac{x-\min(x)}{\max(x)-\min(x)}
$$
The scaling can be implemented with either pure **pandas** or with `MinMaxScaler` from **sklearn**.  

P.S.  
If you want your feature to be in the range `[a, b]` use the following modification:
$$x' = a + \frac{(x - \min(x))(b - a)}{\max(x) - \min(x)}$$

In [ ]:
data = original_data.copy()

In [ ]:
for feature in ("price", "mileage"):
    min_value, max_value = data[feature].min(), data[feature].max()
    data.loc[:, f"{feature}_scaled"] = (data[feature] - min_value) / (max_value - min_value)
    
data

In [ ]:
data["price_scaled"].min(), data["price_scaled"].max()

#### 2) Standard scaling (Normalization)

If you think that your variables might follow a normal distribution, it might be helpful to normalize them so that they have zero-mean and unit variance. This process is called standardization.
$$
x' = \frac{x - \bar{x}}{\sigma}
$$

The scaling can be implemented with either pure **pandas** or with `StandardScaler` from **sklearn**.  

In [ ]:
data = original_data.copy()

In [ ]:
for feature in ("price", "mileage"):
    mean_value, std_value = data[feature].mean(), data[feature].std()
    data.loc[:, f"{feature}_scaled"] = (data[feature] - mean_value) / std_value
    
data

In [ ]:
data["price_scaled"].mean(), data["price_scaled"].std()  # 0 mean and unit variance.

### Handling missing values

One of the first things you might notice when dealing with data in production is that some values are missing. A crucial fact is that not all types of missing values are equal. Let's elaborate on this point. Consider this toy dataset.

| ID | Age | Gender | Annual income | Marital status | Number of children | Job | Buy? |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | ? | A | 150000 | ? | 1 | Engineer | No |
| 2 | 27  | B | 50000 | ? | ? | Teacher | No |
| 3 | ? | A | 100000 | Married | 2 | ? | Yes |
| 4 | 40 | B | ? | ? | 2 | Engineer | Yes |
| 5 | 35  | B | ? | Single | 0 | Doctor | Yes |
| 6 | ? | A | 50000 | ? | 0 | Teacher | No |
| 7 | 33 | B | 60000 | Single | ? | Teacher | No |
| 8 | 20 | B | 10000 | ? | ? | Student | No |


There are 3 types of missing values:
1) **Missing not at random**: when the reason a value is missing is because of the value itself. In this example, we might notice that respondents of gender “B” with higher income tend not to disclose their income (the have reported only low income values). The income values are missing for reasons related to the values themselves.
2) **Missing at random**: when the reason a value is missing is not due to the value itself, but due to another observed variable. In this example, we might notice that age values are often missing for respondents of the gender “A”, which might be because the people of gender A in this survey don’t like disclosing their age.
3) **Missing completely at random**: when there’s no pattern in when the value is missing. In this example, we might think that the missing values for the column “Job” might be completely random, not because of the job itself and not because of another variable. People just forget to fill in that value sometimes for no particular reason. However, this type of missing is very rare. There are usually reasons why certain values are missing, and you should investigate.

When encountering missing values, you can either fill in the missing values with certain values (**imputation**), or remove the missing values (**deletion**).

In [ ]:
data = {
    "ID": [1, 2, 3, 4, 5, 6, 7, 8],
    "Age": ["?", 27, "?", 40, 35, "?", 33, 20], 
    "Gender": ["A", "B", "A", "B", "B", "A", "B", "B"],
    "Annual income": [150000, 50000, 100000, "?", "?", 50000, 60000, 10000],
    "Marital status": ["?", "?", "Married", "?", "Single", "?", "Single", "?"],
    "Number of children": [1, "?", 2, 2, 0, 0, "?", "?"],
    "Job": ["Engineer", "Teacher", "?", "Engineer", "Doctor", "Teacher", "Teacher", "Student"],
    "Buy?": ["No", "No", "Yes", "Yes", "Yes", "No", "No", "No"]
}

df = pd.DataFrame(data)
df

In [ ]:
df.replace("?", np.NaN, inplace=True)  # In order for pandas to recognize missing values.
df

#### 1) Deletion

The simplest way to handle missing values is to delete them. There are two types of deleteion: row-wise and column-wise.

**Column-wise deletion**:
If a variable has too many missing values, just remove that variable. For example, in the example above, over 50% of the values for the variable “Marital status” are missing, so you might be tempted to remove this variable from your model. The drawback of this approach is that you might remove important information and reduce the accuracy of your model. Marital status might be highly correlated to buying houses, as married couples are much more likely to be homeowners than single people.

In [ ]:
df["Marital status"].isna().mean() * 100  # The percentage of missing values in 'Marital status'.

In [ ]:
df.drop(columns=["Marital status"])

**Row-wise deletion**:
If an example has missing value(s), just remove that example from the data. **This method can work when the missing values are completely at random and the number of examples with missing values is small**, such as less than 0.1%. You don’t want to do row deletion if that means 10% of your data examples are removed.  
However, removing rows of data can also remove important information that your model needs to make predictions, especially if the missing values are not at random. For example,  you don’t want to remove examples of gender B respondents with missing income because whether income is missing is information itself (missing income might mean higher income, and thus, more correlated to buying a house) and can be used to make predictions.  
On top of that, removing rows of data can create biases in your model, especially if the missing values are at random. For example, if you remove all examples missing age values in the data, you will remove all respondents with gender A from your data, and your model won’t be able to make predictions for respondents with gender A.

In [ ]:
df.dropna(axis=0, subset=["Job"])

#### 2) Imputation

While it may be tempting to simply delete data that is missing, doing so can result in the loss of crucial information or introduce bias into your model. If you prefer not to delete missing values, you'll need to fill them in through a process known as imputation. However, the difficult part is determining which specific values to use for the imputation.

One common practice is to fill in missing values with their defaults. For example, if the job is missing, you might fill it with an empty string `“”`. Another common practice is to fill in missing values with the mean, median, or mode (the most common value). For example, if the temperature value is missing for a data example whose month value is July, it’s not a bad idea to fill it with the median temperature of July.

In [ ]:
missing_value_map = {
    "Job": ""
}
df.fillna(missing_value_map)

In [ ]:
missing_value_map = {
    "Annual income": df["Annual income"].median()
}
df.fillna(missing_value_map)

#### 3) Bonus 😀 

Impute missing values with one of the imputation methods (e.g. mean imputation) and create a binary column where 1s will repsent missing values. This provides a model an opportunity to decide on handing missing values on its own.

In [ ]:
missing_value_map = {
    "Annual income": df["Annual income"].mean()
}
df["Missing Annual income"] = df["Annual income"].isna().astype(int)
df.fillna(missing_value_map)

**IMPORTANT NOTE**  
Some algorithms, however, support handling missing values inherently. Examples are **XGBoost**, **LightGBM**, **CatBoost**. Moreover, **Catboost** and **LightGBM** can also handle categorical variables (you need to specify categorical variables in the model initiaization step) while **XGBoost** cannot as it expcts only numerical values. 

### **Hyperparameter optimization with** [**Optuna**](https://optuna.org/)

Optuna is a hyperparameter optimization software framework, particularly designed for machine learning pipelines. Optuna enables users to adopt state-of-the-art algorithms for sampling hyperparameters and pruning unpromising trials. This helps to speed up optimization time and performance greatly compared to traditional methods such as GridSearch.

A hyperparameter is a parameter whose value is used to control the learning process (but is not learnt/adjusted during the learning process).

The problem with (hyperparamter) optimization is that, often the search space can be indefinite and the budget to perform this search is constrained. Therefore, to search for the global minimum efficiently, an algorithm has to implement ways to utilize the budget efficiently. The two most commonly used algorithms are the following:  
- **Grid search** - exhaustively searches through a predefined subset.
- **Random search** - selects randomly from a predefined subset.

More advanced family of methods is Bayesian optimization.

**Bayesian optimization**  
Bayesian optimization methods search for global optimization by iteratively building a probabilistic model of the function mapping from hyperparameter values to the objective function. The probabilistic model captures beliefs about the behavior of the function to form a posterior distribution over the objective function. After that, the posterior distribution is further used to form an acquisition function that determines the next point with the best improvement probability.  
Multiple algorithms are then proposed to optimally balance the exploration-exploitation dilemma. Some examples are: *Tree-structured Parzen Estimator (TPE)* and *Gaussian Process Regressor*.

**By default, Optuna implements a Bayesian optimization algorithm (TPE) but it can be easily switched to other supported algorithms in the package**.



In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score
import optuna

In [ ]:
X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {round(accuracy * 100.0, 2)}%")

First of all, Optuna requires us to define an **objective function**.
The objective function will contain the entire logic of a regular model definition, training and testing process. After the model evaluation, it should return the evaluation metric which is also picked by the user.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4, random_state=1)

In [ ]:
def objective(trial):
    
    # 1) Define the hyperparameter set and the corresponding search regions.
    # You can also specify parameters than should be fixed.
    params = {
        "max_depth": trial.suggest_int("max_depth", 1, 9),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500)
    }
    
    # 2) Fit the model.
    optuna_model = XGBClassifier(**params)  # XGBClassifier(max_depth=`suggested value`, ...)
    optuna_model.fit(X_train, y_train)
    
    # 3) Evaluate the model on validation set.
    # In case you data is big and validation set is representative, use it 
    # without cross validation.
    score = cross_val_score(optuna_model, X_val, y_val, cv=3).mean()  # Returns mean accuracy score over folds.
    
    # 4) Return a score summarizing the trial.
    return score

The **Trial** class will be used to store information of one specific combination of hyperparameters later used by the machine learning model.

A **Study** object can then be called to optimize the objective function to find the best hyperparameters combination. It will then run trials iteratively until a user-defined maximum trial or time. The trial with the best hyperparameters will be stored in `study.best_trial`.

In [ ]:
# Create a study.
study = optuna.create_study(direction="maximize")  # We aim to maximize the accuracy (score) returned by objective function.

In [ ]:
# Start optimization.
study.optimize(objective, n_trials=100)

In [ ]:
print(f"Number of finished trials: {len(study.trials)}", "Best trial:", sep="\n")
best_trial = study.best_trial  # Selecting the trial with the highest score.
print(f"  Value: {best_trial.value}", "  Params: ", sep="\n")

for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
optuna.visualization.plot_contour(study, params=["learning_rate", "n_estimators"])

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
# Retrain the model with best hyperparamters.
params = best_trial.params
model = XGBClassifier(**params)
model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

In [ ]:
print(f"Accuracy before: {round(accuracy * 100.0, 2)}%")

y_pred = model.predict(X_test)
accuracy_tuned = accuracy_score(y_test, y_pred)
print(f"Accuracy after: {round(accuracy_tuned * 100.0, 2)}%")

Optuna employs sophisticated hyperparameter optimization algorithms, which are designed to quickly find the optimal objective when it's too expensive to repeatedly train the model. However, it's important to note that when using a less complex model with a small amount of data, GridSearch and RandomSearch may actually outperform these advanced algorithms in terms of both speed and performance. Nevertheless, as the amount of data grows and the models become more intricate, the cost of random hyperparameter tuning increases significantly, making these advanced algorithms highly effective and efficient.

### **Quantization of neural networks**

<center><img src="images/quantization.png" width="500" height="500"/></center>  

[[Image source]](https://developer.nvidia.com/blog/achieving-fp32-accuracy-for-int8-inference-using-quantization-aware-training-with-tensorrt/)

Quantization refers to techniques for doing both computations and memory accesses with lower precision data, usually `int8` compared to floating point implementations. Quantization does not however come without additional cost. Fundamentally quantization means introducing approximations and the resulting networks have slightly less accuracy. These techniques attempt to minimize the gap between the full floating point accuracy and the quantized accuracy.

**The three modes of Quantiation supported in PyTorch starting version 1.3**

1) **Dynamic Quantization**  
This involves not just converting the weights to int8 - as happens in all quantization variants - but also converting the activations to int8 on the fly, just before doing the computation (hence “dynamic”). The computations will thus be performed using efficient int8 matrix multiplication and convolution implementations, resulting in faster compute. However, the activations are read and written to memory in floating point format.

2) **Post-Training Static Quantization**  
One can further improve the performance (latency) by converting networks to use both integer arithmetic and int8 memory accesses. Static quantization performs the additional step of first feeding batches of data through the network and computing the resulting distributions of the different activations (specifically, this is done by inserting “observer” modules at different points that record these distributions). This information is used to determine how specifically the different activations should be quantized at inference time. Importantly, this additional step allows us to pass quantized values between operations instead of converting these values to floats - and then back to ints - between every operation, resulting in a significant speed-up.

3) **Quantization Aware Training (QAT)**  
This method typically results in highest accuracy of these three. With QAT, all weights and activations are “fake quantized” during both the forward and backward passes of training: that is, float values are rounded to mimic int8 values, but all computations are still done with floating point numbers. Thus, all the weight adjustments during training are made while “aware” of the fact that the model will ultimately be quantized; after quantizing, therefore, this method usually yields higher accuracy than the other two methods.

<center><img src="images/quantization_types.png" width="900" height="900"/></center>

[[Source]](https://pytorch.org/blog/introduction-to-quantization-on-pytorch/)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
# Source: https://github.com/pytorch/examples/blob/main/mnist/main.py

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.relu1 = torch.nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.relu2 = nn.ReLU()
        self.flatten = nn.Flatten()
        self.max_pool2d = nn.MaxPool2d(2)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        
        x = self.max_pool2d(x)
        x = self.dropout1(x)
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.relu3(x)
        
        x = self.dropout2(x)
        x = self.fc2(x)
        
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
# Source: https://github.com/pytorch/examples/blob/main/mnist/main.py

def train(model, device, train_loader, optimizer, epoch, log_interval=50):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
# Source: https://github.com/pytorch/examples/blob/main/mnist/main.py

torch.manual_seed(0)
device = torch.device("cpu")

epochs = 1
transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_data = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('../data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    optimizer.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

In [ ]:
model = Net().to(device)
model.load_state_dict(torch.load("mnist_cnn.pt"))
model.eval()

test(model, device, test_loader)

Now let's measure the total inference time of our model over the test data.

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity


with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        test(model, device, test_loader)
        
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=5))

In [ ]:
# Print parameters of our network.
for param_name, param in model.named_parameters():
    print(f"{param_name}: {param.shape}")

In [ ]:
# Choose a weight matrix at random and observe that weight values are all floats.
weight = dict(model.named_parameters())["conv1.weight"]
weight[0][0][0]

In [ ]:
# Moreover, they are stored in 32 bits.
print(f"Weights have a type of {weight.dtype}")

We will apply **Post Training Static Quantization** to our network.  
Post Training Static Quantization (PTQ static) quantizes the weights and activations of the model. It fuses activations into preceding layers where possible. It requires calibration with a representative dataset to determine optimal quantization parameters for activations. Post Training Static Quantization is typically used when both memory bandwidth and compute savings are important with CNNs being a typical use case.

[[Source]](https://pytorch.org/docs/stable/quantization.html#post-training-static-quantization)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.quant = torch.quantization.QuantStub()  # QuantStub converts tensors from floating point to quantized.
        
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.relu1 = torch.nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.relu2 = nn.ReLU()
        self.flatten = nn.Flatten()
        self.max_pool2d = nn.MaxPool2d(2)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
        
        self.dequant = torch.quantization.DeQuantStub()  # DeQuantStub converts tensors from quantized to floating point

        
    def forward(self, x):
        # Manually specify where tensors will be converted from floating point to quantized in the quantized model.
        x = self.quant(x)
        
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.max_pool2d(x)
        x = self.dropout1(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        # Manually specify where tensors will be converted from quantized to floating point in the quantized model.
        x = self.dequant(x)
        
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
# Make sure the network's performane is the same.
model = Net().to(device)
model.load_state_dict(torch.load("mnist_cnn.pt"))
model.eval()
test(model, device, test_loader)

Let's quantize our network!

In [ ]:
model = Net().to(device)
model.load_state_dict(torch.load("mnist_cnn.pt"))
model.eval()  # Model must be set to eval mode for static quantization logic to work.

# Attach a global qconfig, which contains information about what kind of observers to attach. 
# Use 'fbgemm' for server inference and 'qnnpack' for mobile inference.
model.qconfig = torch.quantization.get_default_qconfig("fbgemm")

# Fuse the activations to preceding layers, where applicable. This needs to be done manually depending on the model architecture.
# Common fusions include `conv + relu` and `conv + batchnorm + relu`.
model_fused = torch.quantization.fuse_modules(model, [["conv1", "relu1"], ["conv2", "relu2"], ["fc1", "relu3"]])

# Prepare the model for static quantization. This inserts observers in the model that will 
# observe activation tensors during calibration.
model_prepared = torch.quantization.prepare(model_fused)

# Calibrate the prepared model to determine quantization parameters for activations in 
# a real world setting, the calibration would be done with a representative dataset.
input_fp32, _ = next(iter(train_loader))  # Get the first batch of data.
model_prepared(input_fp32)

# Convert the observed model to a quantized model. This does several things:
# 1) quantizes the weights, 
# 2) computes and stores the scale and bias value to be used with each activation tensor, 
# 3) replaces key operators with quantized implementations.
model_int8 = torch.quantization.convert(model_prepared)

In [ ]:
# Now let's measure the inference time performance of the quantized model.
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        test(model_int8, device, test_loader)
        
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=5))

**We observe significant speed-up in inference time!**

In [ ]:
model_int8

In [ ]:
# Check that weights are converted into 8-bit integers.
print(f"Before: {weight[0][0][0].data}")
print(f"After: {model_int8.conv1.weight().int_repr()[0][0][0].data}")

Some performance results:
<center><img src="images/quantization_results.png" width="900" height="900"/></center>

[[Source]](https://pytorch.org/blog/introduction-to-quantization-on-pytorch/)

# References
- [CS329S: Lecture 4. Feature Engineering](https://docs.google.com/document/d/1N7dRx5zwnyXCl0H-VuKpAHTbMBiZXRHozQ7pScSdz1s/edit)
- [Kaggle tutorial: categorical variables](https://www.kaggle.com/code/alexisbcook/categorical-variables)
- [Working with missing data](https://pandas.pydata.org/docs/user_guide/missing_data.html)
- [Introduction to Quantization in PyTorch](https://pytorch.org/blog/introduction-to-quantization-on-pytorch/)
- [Quantization in PyTorch](https://pytorch.org/docs/stable/quantization.html#module-torch.quantization)
- [A Survey of Quantization Methods for Efficient Neural Network Inference](https://arxiv.org/pdf/2103.13630.pdf)
- [Optuna](https://optuna.org/)